In [2]:
from appgeopy import *
from my_packages import *

# Define the file paths for input and output data
fpath = r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2024_Combined\4__Intersected\InSAR_ModelGPS_Intersect_v2.xlsx"
savefolder = r"E:\SUBSIDENCE_PROJECT_DATA\GPS_2024_Combined\6__Merged\Horizontal"

prepared_los_difference_fpath = r"E:\030_CHOUSHUI_2024\000_INSCALDEFO_2_INSSTACKPSI\PROCESS_005\15_CALIBRATION\PREPARED_InSAR_GPS_LOS_Difference.xlsx"
prepared_los_difference = pd.read_excel(
    prepared_los_difference_fpath, usecols=[0, 1, 2]
)

prepared_los_difference = prepared_los_difference.set_index("STATION")
print(prepared_los_difference.shape)
prepared_los_difference.head(3)

(99, 2)


,X_TWD97,Y_TWD97
STATION,,
ALIS,230933.332644,2.600572e+06
ANES,173423.766349,2.622432e+06
C001,210389.401261,2.590613e+06


In [ ]:
# Load available sheet names from the Excel file
available_sheetnames = data_io.get_sheetnames(fpath)

# ------------------------------------------------------------------------------
# Determine Standard Time Index
# ------------------------------------------------------------------------------
# Find the DataFrame with the maximum number of rows (standard time index)
df_bySheet_maxlength = max(
    pd.read_excel(fpath, sheet_name=sheet).shape[0]
    for sheet in available_sheetnames
)

# Loop through the sheets to find the one with the maximum length to set as standard time index
standard_timeidx = None
for sheet in available_sheetnames:
    df_bySheet = pd.read_excel(
        fpath, sheet_name=sheet, index_col=[0], parse_dates=[0]
    )
    if len(df_bySheet) == df_bySheet_maxlength:
        standard_timeidx = df_bySheet.index
        break

# ------------------------------------------------------------------------------
# Process Data for Each Quantity (N(cm) and E(cm))
# ------------------------------------------------------------------------------
# Loop through the selected quantities (N(cm) and E(cm))
for select_quantity in ["N(cm)", "E(cm)"]:
# for select_quantity in ["N(cm)"]:

    output_records = pd.DataFrame({"STATION": prepared_los_difference.index})

    # Extract X and Y coordinates for each station from the shapefile
    output_records["X_TWD97"] = output_records["STATION"].apply(
        lambda station: prepared_los_difference.query(
            f"STATION == '{station}'"
        )["X_TWD97"].item()
    )
    output_records["Y_TWD97"] = output_records["STATION"].apply(
        lambda station: prepared_los_difference.query(
            f"STATION == '{station}'"
        )["Y_TWD97"].item()
    )

    # Set the index of the DataFrame to the station names
    output_records = output_records.set_index("STATION")

    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

    temp = pd.DataFrame(data=None, index=standard_timeidx)

    # Loop through each sheet and map LOS values to the standard time index
    for select_sheet in available_sheetnames:
        df = pd.read_excel(
            fpath,
            sheet_name=select_sheet,
            usecols=[0, 4, 5, 6],
            index_col=[0],
            parse_dates=[0],
        )
        new_colname = [col.split("_")[-1] for col in df.columns]
        df.columns = new_colname
        temp[select_sheet] = temp.index.map(df[select_quantity])

    # Calculate the difference in displacement values in mm
    difference_in_mm = (temp - temp.shift(1)) * 10
    difference_in_mm = difference_in_mm.drop(labels=df.index[0])

    # Transpose and rename columns
    difference_in_mm_trans = difference_in_mm.T
    difference_in_mm_trans.columns = [
        "N" + col.strftime("%Y%m%d") for col in difference_in_mm_trans.columns
    ]

    # Determine the output file name based on the quantity
    if select_quantity.startswith("N"):
        savename = "PREPARED_NS_Difference_mm_YehDG_HsuYR.xlsx"
    elif select_quantity.startswith("E"):
        savename = "PREPARED_EW_Difference_mm_YehDG_HsuYR.xlsx"
    else:
        print("Error input quantity")
        continue

    # Rename the index and save the final DataFrame to an Excel file
    difference_in_mm_trans = difference_in_mm_trans.rename_axis("STATION")

    for col in difference_in_mm_trans.columns:
        output_records[col] = output_records.index.map(
            difference_in_mm_trans[col]
        )

    savepath = os.path.join(savefolder, savename)
    output_records.to_excel(savepath, index=True)